In [1]:
import mindspore
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset

In [2]:
# Download data from open datasets
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)



file_sizes: 100%|███████████████████████████| 10.8M/10.8M [00:16<00:00, 654kB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [3]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

In [ ]:
print(train_dataset.get_col_names()) # 获取数据集的列名


['image', 'label']


In [5]:
def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0), #将输入的图像缩放为1/255
        vision.Normalize(mean=(0.1307,), std=(0.3081,)), #对图像进行标准化处理
        vision.HWC2CHW() #将图像从HWC格式转换为CHW格式
    ]
    label_transform = transforms.TypeCast(mindspore.int32) #将标签转换为int32类型

    dataset = dataset.map(image_transforms, 'image') 
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size) #设置批处理大小
    return dataset

In [6]:
# Map vision transforms and batch dataset
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)

In [7]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break


Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


网络构建

In [8]:
# Define model
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
print(model)


Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


In [9]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

In [ ]:
# 1. Define forward function
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label) # CrossEntropyLoss expects logits and labels
    return loss, logits

In [11]:
# 2. Get gradient function
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

In [ ]:
# 3. Define function of one-step training
def train_step(data, label):
    (loss, _), grads = grad_fn(data, label) # Get loss and gradients
    optimizer(grads) # Update parameters
    return loss


In [13]:
def train(model, dataset):
    size = dataset.get_dataset_size() # Get the size of the dataset
    model.set_train() # Set the model to training mode
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

In [14]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset)
    test(model, test_dataset, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.292323  [  0/938]
loss: 1.797408  [100/938]
loss: 0.820770  [200/938]
loss: 0.595216  [300/938]
loss: 0.261998  [400/938]
loss: 0.493455  [500/938]
loss: 0.444322  [600/938]
loss: 0.544286  [700/938]
loss: 0.418854  [800/938]
loss: 0.413095  [900/938]
Test: 
 Accuracy: 90.9%, Avg loss: 0.319744 

Epoch 2
-------------------------------
loss: 0.201634  [  0/938]
loss: 0.231194  [100/938]
loss: 0.264057  [200/938]
loss: 0.182378  [300/938]
loss: 0.238462  [400/938]
loss: 0.328668  [500/938]
loss: 0.305571  [600/938]
loss: 0.270367  [700/938]
loss: 0.171518  [800/938]
loss: 0.199179  [900/938]
Test: 
 Accuracy: 92.9%, Avg loss: 0.249662 

Epoch 3
-------------------------------
loss: 0.366589  [  0/938]
loss: 0.257430  [100/938]
loss: 0.207121  [200/938]
loss: 0.291514  [300/938]
loss: 0.169423  [400/938]
loss: 0.266388  [500/938]
loss: 0.134548  [600/938]
loss: 0.344653  [700/938]
loss: 0.224384  [800/938]
loss: 0.231778  [900/938]
Test: 
 

In [16]:
# Save checkpoint
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")


Saved Model to model.ckpt


In [17]:
# Instantiate a random initialized model
model = Network()
# Load checkpoint and load parameter to model
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load, _ = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)


[]


In [18]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break


Predicted: "[9 5 1 0 8 7 6 4 7 2]", Actual: "[7 5 1 2 8 7 6 4 7 2]"
